In [2]:
import pandas as pd
import numpy as np
import geopandas as gp
import libpysal as ps
from mgwr.gwr import GWR, MGWR, GWRResults
from mgwr.sel_bw import Sel_BW


georgia_data = pd.read_csv(ps.examples.get_path('GData_utm.csv'))
georgia_shp = gp.read_file(ps.examples.get_path('G_utm.shp'))

g_y = georgia_data['PctBach'].values.reshape((-1,1))
g_X = georgia_data[['PctFB']].values
u = georgia_data['X']
v = georgia_data['Y']


g_coords = list(zip(u,v))
# georgia_data.head()

#### Standardization Routine

g_X_std = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0) 

g_y_std = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)


g_y = georgia_data['PctBach'].values.reshape((-1,1))
g_X = georgia_data[['PctFB']].values
u = georgia_data['X']
v = georgia_data['Y']

g_coords = list(zip(u,v))

gwr_selector = Sel_BW(g_coords, g_y, g_X)
gwr_bw = gwr_selector.search()
print(gwr_bw)
gwr_results = GWR(g_coords, g_y, g_X, gwr_bw, constant=False).fit()

gwr_results.summary()

155.0
Model type                                                         Gaussian
Number of observations:                                                 159
Number of covariates:                                                     1

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                           7595.738
Log-likelihood:                                                    -532.993
AIC:                                                               1067.986
AICc:                                                              1070.063
BIC:                                                               6794.852
R2:                                                                  -0.481
Adj. R2:                                                             -0.481

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- 

In [3]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy import stats, linalg


scaler = StandardScaler()

x = g_X
y = g_y

# I checked and saw that the results from stdz>>weight for both methods were equivalent. But weight>>stdz is what I need
# to work on. That too should be equivalent as I am doing the estimation for the same location i.e using the same weights. 

wi = gwr_results.model._build_wi(0, gwr_bw).reshape(-1,1)



xw = x*wi
yw = y*wi
xw_std = scaler.fit_transform(xw)
y_std = scaler.fit_transform(yw)

xtw = xw.T   # My thinking is that the weights themselves are standardized values, so after transforming the data by multiplying
             # the x's by the weights one doesn't need to standardize the data anymore.

xtx = np.dot(xtw, xw_std)  # add another variable. make it two


xtx_inv_xt = linalg.solve(xtx, xtw)

betas = np.dot(xtx_inv_xt, y_std)
print(betas)


[[0.60194809]]


In [4]:
gwr_results.params

array([[0.60194809],
       [0.5708762 ],
       [0.59125088],
       [0.58903274],
       [0.79634338],
       [0.84188452],
       [0.84251549],
       [0.8429096 ],
       [0.57064271],
       [0.56186864],
       [0.77938058],
       [0.66262196],
       [0.60757369],
       [0.55966983],
       [0.65771372],
       [0.67741703],
       [0.74718078],
       [0.82703869],
       [0.60875299],
       [0.61584613],
       [0.6641743 ],
       [0.82875859],
       [0.84493274],
       [0.59622296],
       [0.67364276],
       [0.71658954],
       [0.84265712],
       [0.84473517],
       [0.83666624],
       [0.62255353],
       [0.83444839],
       [0.5739676 ],
       [0.84123297],
       [0.57238371],
       [0.56286049],
       [0.79270964],
       [0.55991831],
       [0.82613255],
       [0.76893488],
       [0.59946541],
       [0.84393545],
       [0.84716485],
       [0.57598965],
       [0.83945616],
       [0.60819793],
       [0.63961611],
       [0.5908363 ],
       [0.834

***

***